In [ ]:
import os
import csv
from datetime import date, datetime
import yaml
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from experiment.toolkits.plotting_helper import *

project_dir = Path().resolve().parents[1]
sys.path.insert(0, str(project_dir))
from time import sleep
import pickle
import datetime
from scipy.stats import linregress

In [ ]:
os.chdir(str(project_dir))

In [ ]:
base_cmd = "./bin/modules/wfm-streaming-benchmarking/null-test"
def get_null_prob (wfms_per_segment, num_moves, num_reps):
    command = base_cmd + " " + wfms_per_segment + " " + num_moves + " " + num_reps 
    return os.popen(command).read()

In [ ]:
reps = 1000
num_moves = 16
wfms_per_segment = list(range(16, 1025, 32))

data = []
for i in range(len(num_moves)):
    print(f"{i}/{len(num_moves)}")

    null_prob_str = get_null_prob(wfms_per_segment,
                                i, 
                                reps)
    
    data += [[i, float(null_prob_str)]]


# Save results

In [ ]:
# Saving the results 
file_name = datetime.datetime.now().strftime("%Y-%m-%d, %H-%M-%S.pickle")
with open(os.path.join(str(project_dir), "resources", "wfm-streaming-benchmarking", file_name), "wb") as file:
    pickle.dump(
        {
            "xvals": num_moves,
            "experiment": file_name,
            "wfms_per_segment": wfms_per_segment,
            "data": data 
        }, file
        )

# Load results

In [ ]:
load_latest = True
if not load_latest:
    file_name = "2024-04-22, 19-04-01.pickle"

with open(os.path.join(str(project_dir), "resources", "wfm-streaming-benchmarking", file_name), "rb") as file:
    data = pickle.load(file)
    num_moves = np.array(data["xvals"])
    wfms_per_segment = data["wfms_per_segment"]
    prob_data = data["data"]
    title = data["experiment"]

In [ ]:
acq_time_mean = time_data.mean(axis = 1)
acq_time_std = time_data.std(axis = 1)

fig, ax = getStylishFigureAxes(1, 2, dpi=300)

ErrorbarPlotter(
    fig,
    ax[0],
    roi_heights,
    acq_time_mean / 1e6,
    yerr = acq_time_std / 1e6,
    style = {"marker": "*", "linestyle": "None", "markersize": 1},
    xlim = [-50, 1074],
    ylim = [-5, 85],
    xticks = [0, 256, 512, 768, 1024],
    yticks = [0, 20, 40, 60, 80],
    xlabel = "ROI height (pixels)",
    ylabel = "Acquisition time (ms)"
).draw()

xvals = np.linspace(16, 1024, 100)
yvals = np.array([theoretical_acquistion_time_oct3(W = 1024, H = x) for x in xvals])
PlotPlotter(
    fig,
    ax[0],
    xvals,
    yvals * 1e3,
    style = {"linestyle": "-", "lw": 1, "label": "Theoretical value"},
).draw()

ax[0].axhline(20, linestyle = "--", linewidth = 1, color = "black", label = "Exposure time")
ax[0].legend(fontsize=6)

yvals = np.array([theoretical_acquistion_time_oct3(W = 1024, H = x) for x in roi_heights])
ErrorbarPlotter(
    fig,
    ax[1],
    roi_heights,
    acq_time_mean / 1e6 - yvals * 1e3,
    yerr = acq_time_std / 1e6,
    ylim = [-0.5, 10.5],
    yticks = [0, 2, 4, 6, 8, 10],
    xlim = [-50, 1074],
    xticks = [0, 256, 512, 768, 1024],
    style = {"linestyle": "-", "marker": "s", "markersize": 1},
    xlabel = "ROI height (pixels)",
    ylabel = "$\Delta$t (ms)",
).draw()

plt.suptitle(f"{title.split('.')[0]}\nTime Acquisition Characterization", fontsize=7)
plt.tight_layout()
plt.savefig("result.jpg")
plt.show()

In [ ]:
acq_time_mean = time_data.mean(axis = 1)
acq_time_std = time_data.std(axis = 1)

fig, ax = getStylishFigureAxes(1, 1, dpi=300)

yvals = np.array([theoretical_acquistion_time_oct3(W = 1024, H = x) for x in roi_heights])
ErrorbarPlotter(
    fig,
    ax,
    roi_heights,
    (acq_time_mean / 1e6 - yvals * 1e3) / (yvals * 1e3),
    yerr = acq_time_std / 1e6 / (yvals * 1e3),
    ylim = [-0.02, 0.12],
    yticks = [0, 0.05, 0.1],
    xlim = [-50, 1074],
    xticks = [0, 256, 512, 768, 1024],
    style = {"linestyle": "-", "marker": "s", "markersize": 1},
    xlabel = "ROI height (pixels)",
    ylabel = "$\Delta$t / t${}_{theoretical}$",
).draw()

ax.set_title(f"{title.split('.')[0]}\nTime Acquisition Characterization", fontsize=7)
plt.tight_layout()
plt.savefig("result.jpg")
plt.show()

In [ ]:
acq_time_mean = time_data.mean(axis = 1) / 1e6
thr_acquisition_time = np.array([theoretical_acquistion_time_oct3(H=h, flag_dma=False)*1000 for h in roi_heights])
acq_time_std = time_data.std(axis = 1)

xvals = roi_heights / 1024 * 2
yvals = acq_time_mean - thr_acquisition_time
result = linregress(xvals, yvals)

fig, ax = getStylishFigureAxes(1, 1, axes_list=True, dpi=300)
ErrorbarPlotter(
    fig,
    ax[0],
    xvals,
    yvals,
    yerr = acq_time_std / 1e6,
    style = {"marker": "*", "linestyle": "None", "markersize": 1},
    xlim = [-0.1, 2.1],
    xticks = [0, 1, 2],
    xlabel = "Image size (MB)",
    ylabel = "$t_{transfer}$ (ms)",
    ylim = [-0.5, 10.5],
    yticks = [0, 5, 10],
    title = title.split(".")[0]
).draw()

PlotPlotter(
    fig,
    ax[0],
    xvals,
    xvals * result.slope + result.intercept,
    style = {"linestyle" : "--", "color": "blue", "lw": 1,
             "label": "$f_{PCIe}$ = " + f"{str(round(1 / result.slope, 2))} GB/s"}
).draw()

ax[0].legend(fontsize=6)
plt.savefig("result.jpg")
plt.show()